<a href="https://colab.research.google.com/github/Tobi-KL/mobility_profile_generation/blob/main/Mobility_Profile_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparation

Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load libraries

In [353]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import csv
import xlrd
import random

Read Data

In [ ]:
###   Read data    ###
# data_mop.pkl: Aggregationsmethode "immer die längste Aktivität"
# data_mop_priority: Aggregationsmethode "priority" (immer die Mobilitätsaktivität)

with open('/content/drive/MyDrive/Tobias_MA/Data/data_mop_priority.pkl', 'rb') as input:
        [states_mop,speed_mop,meta_mop,meta_header_mop] = pickle.load(input)

In [ ]:
# nach Jahr 2017 filtern:
meta_mop_filtered = meta_mop[-3820:-1,]
states_mop_filtered = states_mop[-3820:-1,]
speed_mop_filtered = speed_mop[-3820:-1,]

print(meta_mop_filtered)

In [ ]:
state_names = ['not known (mobile or not mobile)', 'work', 'work', 'school,training',
               'shopping', 'private', 'bring,pick up', 'freetime', 'home', 
               'else (outside not mobile)', 'not known (not mobile)', 'by foot', 
               'bicycle', 'motorcycle', 'car driver', 'car codriver', 'public transport', 
               'long distance public-transport', 'else (mobile)', 'not known (mobile)']

no_states = int(len(np.unique(states_mop)))
timesteps = int(states_mop.shape[1])

"""
States:
0:not known (mobile or not mobile)
1:work
2:work
3:school,training
4:shopping
5:private
6:bring,pick up
7:freetime
8:home
9:else (outside not mobile)
10:not known (not mobile)
11:by foot 
12:bicycle
13:motorcycle
14:car driver
15:car codriver
16:public transport
17:long distance public-transport
18:else (mobile)
20:not known (mobile)
"""

'\nStates:\n0:not known (mobile or not mobile)\n1:work\n2:work\n3:school,training\n4:shopping\n5:private\n6:bring,pick up\n7:freetime\n8:home\n9:else (outside not mobile)\n10:not known (not mobile)\n11:by foot \n12:bicycle\n13:motorcycle\n14:car driver\n15:car codriver\n16:public transport\n17:long distance public-transport\n18:else (mobile)\n20:not known (mobile)\n'

In [ ]:
# Infos über die Autos des Haushalts:

# TANK-Tabelle(2017-2018) einlesen:
# SEGMENT in Spalte [174]
# HOUSEHOLD_ID in Spalte [0]

'''
Segmente:

1 Mini
2 Kleinwagen
3 Kompaktklasse
4 Mittelklasse
5 Obere Mittelklasse
6 Oberklasse
7 Geländewagen
8 Sportwagen
9 Mini-Van
10 Großraum-Van
11 Utility
12 Wohnmobil
13 SUV
. Nicht ermittelbar
'''


csv = np.genfromtxt ('/content/drive/MyDrive/Tobias_MA/Data/TANK18.csv', delimiter=";", encoding = "ISO-8859-1")

### Classes

In [ ]:
# erstellt ein Haushalt-Objekt
# enthält Informationen über den gewählten Haushalt (Anzahl Bewohner, Anzahl Fahrer, alle Mobilitätsprofile der Bewohner, Position im Datensatz ... )
# vorerst: entweder zufällig oder mit bekannter ID
# TODO: Eingabemöglichkeit verschiedener Parameter (z.B. Haushaltsgröße, Einkommen etc.) -> Rückgabe einer Liste der ähnlichsten Haushalte (set-Methoden)

class Household:
    def __init__(self, household_ID):
        self.household_ID = household_ID
        #self.household_size = household_size
        self.position = np.where(meta_mop_filtered[:,0] == self.household_ID)[0]
        self.chosen_household = meta_mop_filtered.astype(int) [np.where(meta_mop_filtered[:,0] == self.household_ID)]

        """
        - Constructor of household object
        - householdID: ID of household (MOP-Data) -> sorted in ascending order
        - householdSize: # of occupants of household
        """
  
    def get_household(self):
      return self.chosen_household.astype(int)
    # returns chosen household (array with >= 1 entries (1 entry = 1 person, 1 year))

    def get_number_of_occupants(self):
      return self.chosen_household[0,5]
    # returns number of occupants living in chosen household
    
    def get_number_of_drivers(self):
      states = self.get_states(0,1008) # betrachte states in der ganzen Woche
      number_of_drivers = 0
      for i in range(0,len(states)):
        if ((states[i]==14).sum() > 0):
          number_of_drivers += 1
      return number_of_drivers
    # returns number of drivers (at least one state == 14) living in chosen household

    def get_drivers(self):
      states = self.get_states(0,1008) # betrachte states in der ganzen Woche
      drivers_list = []
      for i in range(0,len(states)):
          if ((states[i]==14).sum() > 0):
            drivers_list.append(self.get_data_position()[i])
      return drivers_list
    # returns list of all drivers of chosen household (nur ihre positions)
    
    def get_number_of_cars(self):
      return self.chosen_household[0,7]
    # returns number of cars of chosen household
    
    def get_data_position(self):
      position_list = []
      for i in self.position:
          position_list.append(i)
      return position_list
    # returns indeces of chosen household in data set
    
    def get_states(self, start, end):
      for i in self.position:
        return states_mop_filtered[self.position,start:end]
    # returns ALL states for timespace between start and end

    def get_adjusted_states(self, start, end):
      states = self.get_states(start,end)
      #states = np.where((states == 8) | (states == 14) | (states == 1) | (states == 2), states, 30)
      return states
    # returns adjusted states
    # 8: home; 1,2: work; 14: Car driver; 30: rest

    def get_speeds(self, start, end):
      for i in self.position:
        return speed_mop_filtered[self.position,start:end]
    # returns all speeds for timespace between start and end

    def get_persons(self, person_number):
      household_positions = self.get_data_position()
      persons = [Person (position = i, household_ID = self.household_ID) for i in household_positions]
      return persons[person_number].position
    # returns position of chosen Person-object in data set

    def get_sorted_drivers(self):
      household_drivers = self.get_drivers()
      drivers = []   
      for i in household_drivers:
        drivers.append(Person(position = i, household_ID = self.household_ID)) #erstellt für jeden driver ein Person-Object
      sorted_drivers = sorted(drivers, key=lambda x: x.total_distance, reverse=True)
      result = []
      for i in range(0,len(sorted_drivers)):
        result.append(sorted_drivers[i].position)
      return result
    # returns list with all drivers in household sorted in descending order by total distance (in whole week)
    ### testen, ob sortieren funktioniert (-> alle sind in MOP bereits absteigend sortiert!)

    def get_sorted_drivers_profiles(self, start, end):
      drivers_list = self.get_sorted_drivers()
      drivers_profiles = np.array([self.get_adjusted_states(start,end)[0]]) 
      for i in range(1, len(drivers_list)):
        drivers_profiles = np.append(drivers_profiles, [self.get_adjusted_states(start,end)[i]], axis=0) 
      return drivers_profiles
    # returns profiles of all drivers (if they don´t have to be merged)

    def get_drivers_states_profiles(self, start, end):
      if self.check_merge()==True:
        drivers_states_profiles = self.states_merge(start,end)
      else:
        drivers_states_profiles = self.get_sorted_drivers_profiles(start,end)
      return drivers_states_profiles
    # returns final states profiles for drivers  

    def check_merge(self):
      if (self.get_number_of_drivers() > self.get_number_of_cars()):
        #print("Merge necessary.")
        return True
      else:
        #print("No merge necessary.")
        return  False
    # checks whether a mobility profile merge is necessary
 
    def states_merge(self, start, end):
    # Drivers müssen bereits sortiert sein!
      x = self.get_number_of_drivers()
      while x > self.get_number_of_cars():
        drivers_positions = self.get_sorted_drivers()      #Liste aller Positionen der Drivers in data
        
        # 2. erstelle Array mit Listen aller states (für jeden driver eine Liste)
        all_states_drivers = np.array([self.get_adjusted_states(start,end)[0]]) 
        for i in range(1, len(drivers_positions)):
          all_states_drivers = np.append(all_states_drivers, [self.get_adjusted_states(start,end)[i]], axis=0)    

        # 3. betrachte ein array aus den letzten beiden Listen
        drivers_last_two = all_states_drivers[-2:]

        # 4. ziehe positions aller 14er aus der letzten Liste heraus
        list_states14_last_driver = [i for i in range(len(drivers_last_two[-1])) if drivers_last_two[-1][i] == 14]

        # 5. erzeuge eine neue Liste (vorletzte Liste komplette + 14er der letzten Liste)
        drivers_last_two_merged = all_states_drivers[-2].copy()
        for i in list_states14_last_driver:
          drivers_last_two_merged[i] = 14
        
        # 6. ersetze die letzten beiden Listen des arrays durch das neue Array
        all_states_drivers = np.delete(all_states_drivers, [-1,-2], axis=0)
        drivers_merged_profiles = np.vstack((all_states_drivers, drivers_last_two_merged))

        x -= 1
      #print("Merge completed.")
      return drivers_merged_profiles
    # returns merged states profiles

    def speed_merge(self, start, end):
    # Drivers müssen bereits sortiert sein!
      x = self.get_number_of_drivers()
      while x > self.get_number_of_cars():
        drivers_positions = self.get_sorted_drivers()      #Liste aller Positionen der Drivers in data
        
        # 2. erstelle Array mit Listen aller states (für jeden driver eine Liste) und ein Array mit Listen aller speeds
        all_states_drivers = np.array([self.get_adjusted_states(start,end)[0]])
        all_speeds_drivers = np.array([self.get_speeds(start,end)[0]]) 
        for i in range(1, len(drivers_positions)):
          all_states_drivers = np.append(all_states_drivers, [self.get_adjusted_states(start,end)[i]], axis=0)
        for i in range(1, len(drivers_positions)):
          all_speeds_drivers = np.append(all_speeds_drivers, [self.get_speeds(start,end)[i]], axis=0)
           

        # 3. betrachte ein array aus den letzten beiden Listen
        drivers_last_two_states = all_states_drivers[-2:]
        drivers_last_two_speeds = all_speeds_drivers[-2:]

        # 4. ziehe positions aller 14er aus der letzten Liste heraus
        list_states14_last_driver = [i for i in range(len(drivers_last_two_states[-1])) if drivers_last_two_states[-1][i] == 14]

        # 5. erzeuge eine neue Liste (vorletzte Liste komplette + 14er der letzten Liste)
        
        drivers_last_two_speeds_merged = all_speeds_drivers[-2].copy()
        for i in list_states14_last_driver:
          drivers_last_two_speeds_merged[i] = all_speeds_drivers[-1][i]
        
        # 6. ersetze die letzten beiden Listen des arrays durch das neue Array
        all_speeds_drivers = np.delete(all_speeds_drivers, [-1,-2], axis=0)
        drivers_merged_speed_profiles = np.vstack((all_speeds_drivers, drivers_last_two_speeds_merged))

        x -= 1
      #print("Merge completed.")
      return drivers_merged_speed_profiles
    # returns merged speed profiles

    def get_drivers_speed(self, start, end): #ohne merge (vgl. states -> umbenennen!)
      drivers_list = self.get_sorted_drivers()
      drivers_profiles = np.array([self.get_speeds(start,end)[0]])    #speeds der ersten person ##[]
      for i in range(1, len(drivers_list)):
        drivers_profiles = np.append(drivers_profiles, [self.get_speeds(start,end)[i]], axis=0) ##[]
      return drivers_profiles
    # returns array with speeds of all drivers

    def get_speeds_after_merge(self,start,end):
      if self.check_merge()==True:
        drivers_speeds_profiles = self.speed_merge(start,end)
      else:
        drivers_speeds_profiles = self.get_drivers_speed(start,end)
      return drivers_speeds_profiles

    def get_drivers_speeds_profiles(self, start, end):
      states = np.array(self.get_drivers_states_profiles(start,end))
      speeds = np.array(self.get_speeds_after_merge(start,end)) 
      mask = np.where((states==14), False, True)      #mit Hilfe der Maske werden alle Speeds, die nicht zu einem 14er state gehören, rausgefiltert
      drivers_speeds_profiles = speeds.copy()
      drivers_speeds_profiles[mask] = 0
      return drivers_speeds_profiles
    # returns final speed profiles of alle drivers (only speeds of states=14)



In [ ]:
# erstellt das Mobilitätsprofil EINER Person
# dafür muss die position der Person im Datensatz angegeben werden (erste Zeile im Datensatz = Person 0)

class Person:
    def __init__(self, position, household_ID):
      self.position = position #position in data
      self.household_ID = household_ID
      #self.total_distance = total_distance
       
      if ((self.get_state(0,1008)==14).sum() > 0):
        self.driver_bool = True  #true, wenn person ein Fahrer ist
      else:
        self.driver_bool = False
      
      if self.driver_bool == True:
        list_1 = self.get_driver_positions(0, 1008) #Liste aller states=14
        speeds_sum = 0
        for i in list_1:
          speeds_sum = speeds_sum + self.get_single_speed(i)     #Summe aller Geschwindigkeiten (state=14) 
        mean_speed = speeds_sum / len(list_1)  # in km/h
        time = len(list_1) * 10 # in min
        distance = mean_speed * time / 60
        self.total_distance = distance
      else:
        self.total_distance = 0   


    """
    - Constructor of Person-Object
    - householdID: ID of household (MOP-Data) -> sorted in ascending order
    - position: position of individual person in data set (first line in data set = person 0)
    """

    def get_state(self, start, end):
        states = states_mop_filtered[self.position,start:end]
        return states
    # returns states for individual persons between start and end

    def get_speed(self, start, end):
        return speed_mop_filtered[self.position,start:end]
    # returns speed for timespace between start and end

    def get_single_speed(self, timestep):
        return speed_mop_filtered[self.position, timestep]
    # returns speed for one single timestep

    def get_adjusted_states(self, start, end):
        states = self.get_state(start,end)
        states_adjusted = np.where((states == 8) | (states == 14) | (states == 1) | (states == 2), states, 30)
        return states_adjusted
    # returns adjusted states for individual person
    # 8: home; 1,2: work; 14: Car driver; 30: rest

    def get_driver_positions(self, start, end):
        list_car_driver = [i for i in range(len(self.get_state(start, end))) if self.get_state(start, end)[i] == 14]
        return list_car_driver
    # returns every position in states where car driver (state=14) = true
    
    def get_total_distance(self, start, end):
      list_1 = self.get_driver_positions(start, end) #Liste aller states=14
      speeds_sum = 0
      for i in list_1:
        speeds_sum = speeds_sum + self.get_single_speed(i)     #Summe aller Geschwindigkeiten (state=14) 
      mean_speed = speeds_sum / len(list_1)  # in km/h
      time = len(list_1) * 10 # in min
      distance = mean_speed * time / 60
      return distance 
    # returns total distance driven by person in the week

In [348]:
class Car:
    def __init__(self, household_ID, car_nr):
      self.household_ID = household_ID
      self.car_nr = car_nr  # number of car in household (1st = 1, 2nd = 2 ... )
    
    """
    - Constructor of Car-Profile Object
    - householdID: ID of household (MOP-Data) -> sorted in ascending order
    """

    def get_speed_profile(self, start, end):
      household_1 = Household(household_ID = self.household_ID)
      speed_profile = household_1.get_drivers_speeds_profiles(start, end)
      return speed_profile [self.car_nr - 1]
    # returns speed profile for chosen car

    def get_state_profile(self, start, end):
      household_1 = Household(household_ID = self.household_ID)
      state_profile = household_1.get_drivers_states_profiles(start, end)
      return state_profile [self.car_nr - 1]
    # returns state profile for chosen car

    def get_segment(self):
      cars = csv [np.where(csv[:,0] == self.household_ID)]
      segment = cars[self.car_nr - 1 : self.car_nr , 174]
      return segment.astype(int)
    # returns segment of chosen car
    
    def get_segments(self):
      cars = csv [np.where(csv[:,0] == self.household_ID)] 
      segments = cars[:,174]
      return segments.astype(int)
    # returns segments of all cars in household (sorted by overall driven distance per year)
    
    def get_charging_options(self, start, end, home_charging_power, work_charging_power):
      states = self.get_state_profile(start,end)
      #charge_array = states.copy()
      charge_array = np.where((states == 8), home_charging_power, np.where((states == 1) | (states == 2), work_charging_power, np.where((states == 14), -100, 0))) 
      return charge_array
    # returns array with charging options (considers charging power at home, at work; 0 when not in use (but neither at home or work), -100 when in use)

# Informationen, die Ladeprofil enthalten soll: Heinz S. 44 


In [349]:
ID = 4711036120
household_1 =  Household(household_ID = ID)

start = 588
end = 732

home_charging_power = 10
work_charging_power = 20

car_1 = Car(ID, 1)
print(car_1.get_charging_options(start,end,home_charging_power,work_charging_power))

[  10   10   10   10   10   10   10   10   10   10   10   10   10   10
   10   10   10   10   10   10   10   10   10   10   10   10   10   10
   10   10   10   10   10   10   10   10   10   10   10   10   10   10
   10   10   10   10   10   10   10   10    0    0    0    0   20   20
   20   20   20   20    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   10   10   10   10   10   10   10   10   10   10   10
   10   10   10   10   10   10   10   10   10 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100   10   10   10   10   10   10   10   10
   10   10   10   10   10   10   10   10   10   10   10   10   10   10
   10   10   10   10]


### Tool

In [ ]:
'''
*   Bsp. für HH mit 1 Fahrer,  1 Auto:  4301011562
*   Bsp. für HH mit 2 Fahrern, 2 Autos: 4711036120
*   Bsp. für HH mit 3 Fahrern, 3 Autos: 4301011674
*   Bsp. für HH mit 2 Fahrern, 1 Auto:  4301011871
*   Bsp. für HH mit 0 Fahrern, 1 Auto:  4301012968
'''

### Input ###

ID = 4711036120

start = 588
end = 732

home_charging_power = 10
work_charging_power = 20

### Input end ###

household_1 =  Household(household_ID = ID)
print('number of drivers in chosen household: ', household_1.get_number_of_drivers())
print('number of cars in chosen household: ', household_1.get_number_of_cars(),'\n')
print("Timesteps", start, "until", end, "\n")
print("\n")

for i in range(household_1.get_number_of_cars()):
  car_1 = Car(household_ID = ID, car_nr = i+1)
  print("Car Segment of car", i+1, ":", car_1.get_segment(),'\n')
  print("States Profile of car", i+1, ":\n", car_1.get_state_profile(start,end),'\n')
  print("Speed Profile of car", i+1, ":\n", car_1.get_speed_profile(start,end),'\n')
  print("\n")

# Charging options:
print("Charging options:\n")
for i in range(household_1.get_number_of_cars()):
  print(car_1.get_charging_options(start,end,home_charging_power,work_charging_power))
  print("\n")

# Speed Plots:
print("Speed plots:\n")
for i in range(household_1.get_number_of_cars()):
  car_1 = Car(household_ID = ID, car_nr = i+1)
  plt.figure(i)
  plt.plot(car_1.get_speed_profile(start,end))
  

In [ ]:
'''
*   Bsp. für HH mit 1 Fahrer,  1 Auto:  4301011562
*   Bsp. für HH mit 2 Fahrern, 2 Autos: 4711036120
*   Bsp. für HH mit 3 Fahrern, 3 Autos: 4301011674
*   Bsp. für HH mit 2 Fahrern, 1 Auto:  4301011871
*   Bsp. für HH mit 0 Fahrern, 1 Auto:  4301012968
'''

household_1 =  Household(household_ID = 4301011674)

start = 0
end = 144

x = household_1.get_drivers_states_profiles(start,end)
y = household_1.get_drivers_speeds_profiles(start,end)

print('\nDriver Profiles of household:\n', x)
print('\nSpeed Profiles of household:\n', y)

In [ ]:
# Erstellung eines Haushalt-Objekts in meta_mop_filtered
# ansonsten: gewünschte ID eingeben

household_1 =  Household(household_ID = 4301011674)
#household_1.get_household()

In [ ]:
# Anzahl Bewohner des Haushalts
# Anzahl Fahrer im Haushalt
# Anzahl Autos im Haushalt

print('number of occupants in chosen household: ', household_1.get_number_of_occupants())
print('number of drivers in chosen household: ', household_1.get_number_of_drivers())
print('number of cars in chosen household: ', household_1.get_number_of_cars())

In [ ]:
# Informationen über die Position des Haushalts bzw. der Personen im Datensatz (filtered: nur 2017)
# gibt die Position der gewünschten Person im Datensatz an. (0 = erste Person des HH usw.)

print('The household can be found in the data set at row(s): ', household_1.get_data_position())

person_number = 0     # 0: first person, 1: second person ...
print('Person', person_number, 'can be found in the data set at row: ', household_1.get_persons(person_number))

# gebe drivers aus:
print('Drivers of household:', household_1.get_drivers())

In [ ]:
# Ausgabe der Mobilitätsprofile

start = 0    # timesteps
end = 504
person_number = 0

mobility_profiles = household_1.get_states(start,end)

print('Mobility profile of person', person_number, ':')
plt.plot(mobility_profiles[person_number])

In [ ]:
# Ausgabe der Speed-Profiles

start = 0     # timesteps
end = 504
person_number = 0

speed_profiles = household_1.get_speeds(start, end)

print('Speed profile of person', person_number, ':')
plt.plot(speed_profiles[person_number])

In [ ]:
# Ausgabe der States eines individuellen HH-Mitglieds
person_number = 0
household_ID = household_1.household_ID
start = 0
end = 504


# erstelle Person Objekt:
person_1 = Person (position = household_1.get_persons(person_number), household_ID = household_ID)
print('original states for person', person_number,':\n', person_1.get_state(start,end))
print('\n')

# setze alle states außer 1,2 (work), 8 (home), 14 (car driver) auf 30??
print('adjusted states for person', person_number, ':\n', person_1.get_adjusted_states(start, end))
print('\n')

print('Person', person_number, 'is a driver:', person_1.driver_bool)
print('Person', person_number, 'drove the following total distance (km) in one weeek:', person_1.total_distance)

# Liste aller Positionen "Car Driver":
print('positions where car driver is true for person', person_number,':\n', person_1.get_driver_positions(start, end))

### Test

In [ ]:
# Erklärung Funktion merge()

household_1 =  Household(household_ID = 4301011871)

# Funktioniert nur für Haushalte mit mind. 2 Drivers (vorher check laufen lassen)
# 1. positions aller driver des HH
drivers_positions = household_1.get_drivers()
print('Drivers:\n', drivers_positions)
print('-----')

# 2. erstelle Array mit Listen aller states (für jeden driver eine Liste)
all_states_drivers = np.array([household_1.get_states(0,14)[0]])
for i in range(1, len(drivers_positions)):
  all_states_drivers = np.append(all_states_drivers, [household_1.get_states(0,14)[i]], axis=0)
print('States aller Drivers:\n', all_states_drivers)
print('-----')

# 3. betrachte ein array aus den letzten beiden Listen
drivers_last_two = all_states_drivers[-2:]
print('Die letzten beiden Einträge des Arrays:\n', drivers_last_two)
print('-----')

# 4. ziehe positions aller 14er aus der letzten Liste heraus
list_states14_last_driver = [i for i in range(len(drivers_last_two[-1])) if drivers_last_two[-1][i] == 14]
print('Alle 14er states aus dem LETZTEN Eintrag:\n', list_states14_last_driver)
print('-----')

# 5. erzeuge eine neue Liste (vorletzte Liste komplette + 14er der letzten Liste)
drivers_last_two_merged = all_states_drivers[-2].copy()
for i in list_states14_last_driver:
  drivers_last_two_merged[i] = 14
print('VORletzter Eintrag, ergänzt mit den 14ern des LETZTEN Eintrags:\n', drivers_last_two_merged)
print('-----')

# 6. ersetze die letzten beiden Listen des arrays durch das neue Array
all_states_drivers = np.delete(all_states_drivers, [-1,-2], axis=0)
drivers_merged = np.vstack((all_states_drivers, drivers_last_two_merged))
print('Merge abgeschlossen. Das folgende Array hat die letzten beiden Drivers zusammengefasst: besteht aus den States des vorletzten Drivers, ergänzt durch die 14er des letzten Drivers:\n', drivers_merged)